In [1]:
import cv2
import numpy as np
import os
from scipy.optimize import least_squares
# from tomlkit import boolean
from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from data import Image_loader
import lib

Load Images

In [2]:
img_obj = Image_loader("Datasets\\castle-P30")
enable_bundle_adjustment = False

Finding keypoints from 2 images

In [3]:
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
pose_array = img_obj.K.ravel()
transform_matrix_0 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])
transform_matrix_1 = np.empty((3, 4))

pose_0 = np.matmul(img_obj.K, transform_matrix_0)
pose_1 = np.empty((3, 4)) 
total_points = np.zeros((1, 3))
total_colors = np.zeros((1, 3))

image_0 = img_obj.downscale_image(cv2.imread(img_obj.image_list[0]))
image_1 = img_obj.downscale_image(cv2.imread(img_obj.image_list[1]))

feature_0, feature_1 = lib.find_keypoints(image_0, image_1)

Calculating Essential and Rotation Matrixes

In [4]:
essential_matrix, em_mask = cv2.findEssentialMat(feature_0, feature_1, img_obj.K, method=cv2.RANSAC, prob=0.999, threshold=0.4, mask=None)
feature_0 = feature_0[em_mask.ravel() == 1]
feature_1 = feature_1[em_mask.ravel() == 1]


_, rot_matrix, tran_matrix, em_mask = cv2.recoverPose(essential_matrix, feature_0, feature_1, img_obj.K)
feature_0 = feature_0[em_mask.ravel() > 0]
feature_1 = feature_1[em_mask.ravel() > 0]
transform_matrix_1[:3, :3] = np.matmul(rot_matrix, transform_matrix_0[:3, :3])
transform_matrix_1[:3, 3] = transform_matrix_0[:3, 3] + np.matmul(transform_matrix_0[:3, :3], tran_matrix.ravel())

Triangulation

In [5]:
pose_1 = np.matmul(img_obj.K, transform_matrix_1)

feature_0, feature_1, points_3d = lib.triangulation(pose_0, pose_1, feature_0, feature_1)

Calculating reprojection error

In [6]:
error, points_3d = lib.reprojection_error(points_3d, feature_1, transform_matrix_1, img_obj.K, is_homogeneous = True)
print("REPROJECTION ERROR: ", error)


REPROJECTION ERROR:  0.006572311839443069


PnP

In [7]:
_, _, feature_1, points_3d, _ = lib.PnP(points_3d, feature_1, img_obj.K, np.zeros((5, 1), dtype=np.float32), feature_0, is_initial=True)

TypeError: PnP() got an unexpected keyword argument 'initial'

: 

In [ ]:
total_images = len(img_obj.image_list) - 2 
pose_array = np.hstack((np.hstack((pose_array, pose_0.ravel())), pose_1.ravel()))

Looping through images and ca

In [ ]:
threshold = 0.5
for i in tqdm(range(total_images)):
    image_2 = img_obj.downscale_image(cv2.imread(img_obj.image_list[i + 2]))
    features_cur, features_2 = lib.find_keypoints(image_1, image_2)

    if i != 0:
        feature_0, feature_1, points_3d = lib.triangulation(pose_0, pose_1, feature_0, feature_1)
        feature_1 = feature_1.T
        points_3d = cv2.convertPointsFromHomogeneous(points_3d.T)
        points_3d = points_3d[:, 0, :]
    

    cm_points_0, cm_points_1, cm_mask_0, cm_mask_1 = lib.matching_points(feature_1, features_cur, features_2)
    cm_points_2 = features_2[cm_points_1]
    cm_points_cur = features_cur[cm_points_1]

    rot_matrix, tran_matrix, cm_points_2, points_3d, cm_points_cur = lib.PnP(points_3d[cm_points_0], cm_points_2, img_obj.K, np.zeros((5, 1), dtype=np.float32), cm_points_cur, is_initial = False)
    transform_matrix_1 = np.hstack((rot_matrix, tran_matrix))
    pose_2 = np.matmul(img_obj.K, transform_matrix_1)

    error, points_3d = lib.reprojection_error(points_3d, cm_points_2, transform_matrix_1, img_obj.K, is_homogeneous = False)

    
    cm_mask_0, cm_mask_1, points_3d = lib.triangulation(pose_1, pose_2, cm_mask_0, cm_mask_1)
    error, points_3d = lib.reprojection_error(points_3d, cm_mask_1, transform_matrix_1, img_obj.K, is_homogeneous = True)
    print("Reprojection Error: ", error)
    pose_array = np.hstack((pose_array, pose_2.ravel()))

    if enable_bundle_adjustment:
        points_3d, cm_mask_1, transform_matrix_1 = lib.bundle_adjustment(points_3d, cm_mask_1, transform_matrix_1, img_obj.K, threshold)
        pose_2 = np.matmul(img_obj.K, transform_matrix_1)
        error, points_3d = lib.reprojection_error(points_3d, cm_mask_1, transform_matrix_1, img_obj.K, is_homogeneous = False)
        print("Bundle Adjusted error: ",error)
        total_points = np.vstack((total_points, points_3d))
        points_left = np.array(cm_mask_1, dtype=np.int32)
        color_vector = np.array([image_2[l[1], l[0]] for l in points_left])
        total_colors = np.vstack((total_colors, color_vector))
    else:
        total_points = np.vstack((total_points, points_3d[:, 0, :]))
        points_left = np.array(cm_mask_1, dtype=np.int32)
        color_vector = np.array([image_2[l[1], l[0]] for l in points_left.T])
        total_colors = np.vstack((total_colors, color_vector)) 
    



    transform_matrix_0 = np.copy(transform_matrix_1)
    pose_0 = np.copy(pose_1)

    image_0 = np.copy(image_1)
    image_1 = np.copy(image_2)
    feature_0 = np.copy(features_cur)
    feature_1 = np.copy(features_2)
    pose_1 = np.copy(pose_2)
    cv2.imshow(img_obj.image_list[0].split('\\')[-2], image_2)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
cv2.destroyAllWindows()


  4%|▎         | 1/28 [00:00<00:18,  1.45it/s]

Reprojection Error:  1.250388848252236


  7%|▋         | 2/28 [00:01<00:19,  1.35it/s]

Reprojection Error:  2.9059230243830276


 11%|█         | 3/28 [00:02<00:19,  1.30it/s]

...

Reprojection Error:  0.5139078080868782


 96%|█████████▋| 27/28 [00:19<00:00,  1.47it/s]

Reprojection Error:  0.8692902450162762


100%|██████████| 28/28 [00:20<00:00,  1.38it/s]

Reprojection Error:  2.548209137828322


Saving Results

In [ ]:
print(total_points.shape, total_colors.shape)
img_obj.to_ply(total_points, total_colors)
print("Completed Exiting ...")
np.savetxt(img_obj.path + '\\res\\' + img_obj.image_list[0].split('\\')[-2]+'_pose_array.csv', pose_array, delimiter = '\n')

(19869, 3) (19869, 3)
(19869, 3) (19869, 3)
Completed Exiting ...
